In [1]:
import rosbag
import tf
from geometry_msgs import msg
import geometry_msgs.msg

import tf_conversions.posemath as posemath
import numpy as np

In [2]:
# input_bag = "/home/ruffsl/Desktop/data_02_18/kk_cup.bag"
# output_bag = "/home/ruffsl/Desktop/data_02_18/optical.kk_cup.bag"
centroid2optic = np.matrix([[0, 0, 1, 0],
                          [-1, 0, 0, 0],
                          [0, -1, 0, 0],
                          [0, 0, 0, 1]])
# centroid2optic = np.matrix([[1, 0, 0, 0],
#                           [0, 1, 0, 0],
#                           [0, 0, 1, 0],
#                           [0, 0, 0, 1]])

http://www.euclideanspace.com/maths/geometry/rotations/conversions/eulerToMatrix/examples/index.htm

In [3]:
def applyTF2bag(input_bag, output_bag):
    isFirstTF = True
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(input_bag).read_messages():
            # This also replaces tf timestamps under the assumption 
            # that all transforms in the message share the same timestamp
            if topic == "/tf":

                tf_msg = tf.msg.tfMessage()

                for geo_msg in msg.transforms:
                    if geo_msg.child_frame_id == "Robot_1/base_link":
                        geo_msg.header.frame_id = "odom"

                        q = np.zeros(4)
                        q[0] = geo_msg.transform.rotation.x
                        q[1] = geo_msg.transform.rotation.y
                        q[2] = geo_msg.transform.rotation.z
                        q[3] = geo_msg.transform.rotation.w
                        P = tf.transformations.quaternion_matrix(q)
                        P[0, 3] = geo_msg.transform.translation.x
                        P[1, 3] = geo_msg.transform.translation.y
                        P[2, 3] = geo_msg.transform.translation.z
                        P_ = P*centroid2optic

                        if isFirstTF:
                            isFirstTF = False
                            firstTF = np.linalg.inv(P_)
                        P_ = firstTF*P_

                        q = tf.transformations.quaternion_from_matrix(P_)

                        geo_msg.transform.rotation.x = q[0]
                        geo_msg.transform.rotation.y = q[1]
                        geo_msg.transform.rotation.z = q[2]
                        geo_msg.transform.rotation.w = q[3]
                        geo_msg.transform.translation.x = P_[0, 3]
                        geo_msg.transform.translation.y = P_[1, 3]
                        geo_msg.transform.translation.z = P_[2, 3]
                        tf_msg.transforms.append(geo_msg)
                        outbag.write('/tf', tf_msg, msg.header.stamp if msg._has_header else t)
#                     else:
#                         tf_msg.transforms.append(geo_msg)

#                 outbag.write('/tf', tf_msg, msg.header.stamp if msg._has_header else t)
            else:
                outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)

In [5]:
bagfiles = ['kk_cup.bag',
'ojflo_data.bag',
'ojflo_fast.bag',
'ojflo_occ.bag',
'ron_data.bag',
'ron_fast.bag',
'ron_occ.bag',
'ss_data.bag',
'ss_occ.bag',
'tide.bag',
'tide_fast.bag',
'tide_occ.bag']
path = '/media/ruffsl/Seagate Backup Plus Drive/data_tracking_0218/data_02_18/'
for bagfile in bagfiles:
    applyTF2bag(path+bagfile, path+'optical/optical.'+bagfile)